# Load Model, Dataset

In [ ]:
from google.colab import drive
import json
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
with open("/content/drive/My Drive/sharded_dataset.json") as f:
  data = json.load(f)

In [ ]:
API = []
for task in data:
  if "actions" in task['task']:
    API.append(task)

# OPENAI CODE

In [ ]:
from openai import OpenAI
import math

def predictive_entropy_uncertainty_chat(prompt,
                                         temperature=1.0,
                                         max_tokens = 500,
                                         logprobs= True):
    """
    Send `prompt` to OpenAI Completion API, return average token entropy and generated text.
    Entropy is approximated from top `logprobs` returned per token.
    """

    client = OpenAI(api_key="FILLER")
    resp = client.chat.completions.create(
        model="gpt-4.1",
        messages=prompt,
        max_completion_tokens=max_tokens,
        temperature=temperature,
        logprobs=logprobs,
        top_logprobs = 20,
    )

    generated_tokens = resp.choices[0].message.content
    lps = resp.choices[0].logprobs.content

    entropies = []

    for token_info in lps:
        entropy = 0.0
        for alt in token_info.top_logprobs:
            p = math.exp(alt.logprob)
            entropy += -p * alt.logprob
        entropies.append(entropy)

    avg_entropy = sum(entropies) / len(entropies) if entropies else 0.0

    tokens_used = resp.usage.completion_tokens + resp.usage.prompt_tokens

    return avg_entropy, generated_tokens, tokens_used

In [ ]:
import re
import os

def prompt_rewrite(prompt):

  new_prompt = [
    {
        "role": "system",
        "content": "You are a prompt rewriter whose main goal is to rewrite the prompt given by the user in the most optimal way without losing any information in them."
    },
    {
        "role": "user",
        "content": (
            "I have a set of questions and/or statements, please REWRITE all the questions/statements so that they are in the most optimal order that is the easiest to understand. DO NOT ANSWER ANY OF THE QUESTIONS JUST REWRITE. Here are the instructions:\n"
            "how much will John pay in sales tax?\n"
            "the purchase amount is $120\n"
            "the sales tax rate is 8.25%."
        )
    },
    {
        "role": "assistant",
        "content": "What is the total sales tax John will pay on a $120 purchase at an 8.25% rate?"
    },
]

  user_content = "I have a set of questions and/or statements, please REWRITE all the questions/statements so that they are in the most optimal order that is the easiest to understand. DO NOT ANSWER ANY OF THE QUESTIONS JUST REWRITE. Here are the instructions:\n"
  user_messages = [item["content"] for item in prompt if item.get("role") == "user"]
  for msg in user_messages:
    user_content += msg + "\n"

  new_prompt.append({"role": "user", "content": user_content})

  return new_prompt

In [ ]:
import random

def with_context_reset_chat(dataset, file_path, threshold,
                            temperature=1.0, runs=1, numQ=50):

    connectors = ["oh also, ", "I just remembered, ", "sorry i forgot to say, ", "", "oh, and ", "FYI, "]
    tokens_used = 0

    for run in range(runs):
        print(f"Run {run+1}/{runs}")
        out_path = file_path.replace(".json", f"_run{run}.json")
        results = []

        for entry in dataset:
            base_system = {
        "role": "system",
        "content": (
            f"""You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out.
You should only return the function calls in your response.

If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.

At each turn, you should try your best to complete the tasks requested by the user within the current turn. Continue to output functions to call until you have fulfilled the user's request to the best of your ability. Once you have no more functions to call, the system will consider the current turn complete and proceed to the next turn or task.

Here is a list of functions in JSON format that you can invoke.

{entry['function']}"""
        )
    }
            shards = entry["shards"]
            print(f"Question with {len(shards)} shards")
            messages = [base_system]
            prev_entropy = float("inf")
            resets = 0
            entropies = []
            before_reset = None
            choice = random.choice(connectors)

            for shard in shards:
                user_content = shard["shard"]
                if shard['shard_id'] != 1:
                  user_content = choice + user_content
                if shard["shard_id"] == len(shards):
                    user_content += " Please include all the functions neccessary to complete my task in your response."
                messages.append({"role": "user", "content": user_content})

                entropy, reply, tok = predictive_entropy_uncertainty_chat(messages, temperature=1.0, logprobs=True)
                print(f"Entropy: {entropy:.4f}")
                tokens_used += tok

                if entropy - prev_entropy > threshold:
                    before_reset = list(messages)
                    messages = prompt_rewrite(messages)
                    entropy, reply, tok = predictive_entropy_uncertainty_chat(messages, temperature=0.2, logprobs=True)
                    tokens_used += tok
                    messages = [base_system, {"role": "user", "content": reply}]
                    entropy, reply, tok = predictive_entropy_uncertainty_chat(messages, temperature=1.0, logprobs=True)
                    print(f"Reset entropy: {entropy:.4f}")
                    tokens_used += tok
                    resets += 1

                prev_entropy = entropy
                entropies.append(entropy)
                messages.append({"role": "assistant", "content": reply})

                print(f"Current Tokens Used: {tokens_used}")

                if shard["shard_id"] == len(entry["shards"]):

                  if before_reset:
                    chat_history = f"{before_reset}\n\nAFTER RESET\n\n{messages}"
                  else:
                    chat_history = messages

                  new_entry = {"final_output": reply, "chat_history": chat_history, "entropies": entropies, "resets":resets}

                  if os.path.exists(out_path):
                    with open(out_path, "r") as f:
                        data = json.load(f)
                  else:
                      data = []

                  data.append(new_entry)

                  with open(out_path, "w") as f:
                      json.dump(data, f, indent=2)